# Single-Qubit Noisy Simulator


*Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## Outline

 This tutorial describes how to use Quanlse's single-qubit noisy simulator to define the decoherence noise and amplitude noise and simulate the gate operation. The outline of this tutorial is as follows:
 
- Introduction
- Preparation
- Define simulator parameters and pulses
- $\pi$ pulse calibration using Rabi oscillation 
- Measure relaxation time $T_1$ 
- Measure decoherence time $T_2$ by Ramsey experiment
- Single-qubit noisy simulator at the gate level
- Summary

## Introduction

At the pulse level, a single-qubit's control process requires applying pulses to the qubit. In this tutorial, the single-qubit noisy simulator enables the simulation of the dynamical evolution of the control based on the control pulse and noise parameters input by the user.

The system Hamiltonian of a three-level superconducting qubit in the rotating frame after RWA (Rotating Wave Approximation) can be written as \[1\]:

$$
\hat{H}_{\rm sys}(t) = \hat{H}_{\rm anharm} + \hat{H}_{\rm noise}(t) + \hat{H}_{\rm ctrl}(t),
$$

where the anharmonicity term $\hat{H}_{\rm anharm}$ is related to qubit's anharmonicity parameter $\alpha$:

$$
\hat{H}_{\rm anharm} = \frac{\alpha}{2} \hat{a}^\dagger \hat{a}^\dagger \hat{a}\hat{a},
$$

where $\hat{a}^\dagger$ and $\hat{a}$ are the three-level qubit creation operator and annihilation operator, respectively. In the single-qubit noisy simulator, we introduce two kinds of noise: decoherence noise due to the environment and amplitude noise due to fluctuations of the control pulse waveform $\hat{H}_{\rm amp}$ \[1\]:

$$
\hat{H}_{\rm amp}(t) = \epsilon \hat{H}_{\rm ctrl},
$$

where the probability distribution of the amplitude noise parameter $\epsilon$ follows Gaussian distribution:

$$
P(\epsilon) = \frac{1}{\sqrt{ 2 \pi \sigma^2 }}e^{-\frac{ \epsilon^2} {2 \sigma^2} }.
$$

The parameters $T_1$ and $T_2$ related to decoherence describe the decay rate of non-diagonal elements of the density matrix and the population decay rate on the excited state. We can use Bloch-Redfield density matrix $\rho_{BR}$ to illustrate the decoherence process of an initialized qubit $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$ under open-system evolution ($\delta \omega$ is the difference between the frequency of the control pulse and the frequency of the qubit) \[2\]:

$$
\rho_{BR} =
\begin{pmatrix}
1 + (|\alpha|^2-1)e^{-t/T_1} & \alpha\beta^* e^{i\delta \omega t}e^{-t/T_2} \\
\alpha^* \beta e^{-i\delta\omega t} e^{-t/T_2} & |\beta|^2 e^{-t/T_1}
\end{pmatrix}.
$$

As it shows, the decay rate of the population on the excited state is related to $T_1$, and the decay rate of the non-diagonal elements is related to $T_2$.

Therefore, we use the coefficients $\sigma$, $T_1$, and $T_2$ to characterize these two kinds of noise, which means the user can change the three parameters to study the noises of the system.

## Preparation

After you have successfully installed Quanlse, you could run the Quanlse program below following this tutorial. To run this particular tutorial, you would need to import the following packages from Quanlse and other commonly-used Python libraries:

In [ ]:
# Import 1-qubit noisy simulator at the pulse level
from Quanlse.Superconduct.Simulator import PulseModel
from Quanlse.Superconduct.Simulator.PulseSim1Q import pulseSim1Q
from Quanlse.QOperator import driveX, driveZ
from Quanlse.QWaveform import gaussian, square
from Quanlse.Utils.Functions import basis, dagger, expect, project
from Quanlse.QOperation.FixedGate import H, X, Y, Z
from Quanlse.QOperation.RotationGate import RX, RY, RZ
from Quanlse.Utils.Bloch import plotBloch, rho2Coordinate
from Quanlse.remoteSimulator import remoteSimulatorRunHamiltonian as runHamiltonian

# Import tool for analysis
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
from math import pi
import numpy as np

To use the Quanlse Cloud Service, we need to acquire a token from http://quantum-hub.baidu.com to get access to the cloud. 

In [ ]:
# Import Define class and set the token for cloud service
# Please visit http://quantum-hub.baidu.com
from Quanlse import Define
Define.hubToken = ''

## Define simulator parameters and pulses

We first define the required parameters of the simulator.

In [ ]:
qubitNum = 1  # The number of qubits
dt = 0.2  # The sampling period, in nanoseconds
level = 3  # The energy level
anharm = -0.3472 * (2 * pi)  # The anharmonicity of the qubit, in 2 * pi * GHz
wq = 4.9  * (2 * pi)  # The qubit frequency, in 2 * pi * GHz

# Define the noise of the simulator
ampSigma = 0.02  # amplitude (over-rotation) error
t1 = 3000  # qubit relaxation time, in nanoseconds
t2 = 800  # qubit dephasing time, in nanoseconds

Create an object of class `PulseModel`. The object is a physics model defined by the parameters above.

In [ ]:
qubitAnharm = {0: anharm}
qubitFreq  = {0: wq}
qubitT1 = {0: t1}
qubitT2 = {0: t2}

model = PulseModel(subSysNum=qubitNum, sysLevel=level, dt=dt, ampSigma=ampSigma,
                   T1=qubitT1, T2=qubitT2, qubitFreq=qubitFreq, qubitAnharm=qubitAnharm)

ham = model.createQHamiltonian()

We have constructed the physics model of the simulator. Then we can use Rabi oscillation to find the amplitudes of $\pi$ and $\pi / 2$ pulse.

## $\pi$ pulse calibration using Rabi oscillation

In this section, we use Rabi oscillation to calibrate $\pi$ pulse. Namely, we fix the duration time of the pulse, then vary the amplitude of the driving pulse and record the population of state $|1\rangle$. This process can be done by the single-qubit simulator.

In [ ]:
# Define the amplitudes of the pulse
ampList = np.linspace(0, 0.4, 100)

# Create a jobList object
rabiJob = ham.createJobList()

# Define the shape of the gaussian waveform
tg = 60
tau = tg / 2
sigma = tg / 8

# Append each job of different pulse amplitudes to jobList
for amp in ampList:
    wave = gaussian(t=tg, a=amp, tau=tau, sigma=sigma)
    job = ham.createJob()
    job.appendWave(operators=driveX, onSubSys=0, waves=wave)
    job = model.getSimJob(job)
    rabiJob.addJob(job)

After defining the `jobList` of Rabi oscillation, we call the module `runHamiltonian()` with input initial state $|\psi\rangle = |0\rangle$ and the joblist. We set the parameter `isOpen=True` to simulate the evolution in the open system.

In [ ]:
# Define the initial state for the simulation
stateInit = basis(level, 0) 
    
# Run the simulation of the open system evolution
result = runHamiltonian(ham=ham, state0=stateInit, jobList=rabiJob, isOpen=True)

Then we plot the population on state $|1\rangle$ as a function of the driving pulse amplitude.

In [ ]:
# Define the projector
prj = basis(level, 1) @ dagger(basis(level, 1))

popList = []

# Compute the population for each job
for res in result:
    rho = res['state']
    popList.append(expect(prj, rho))  # Compute the population of |1>

plt.plot(ampList, popList, '.')
plt.xlabel('Amplitudes (a.u.)')
plt.ylabel(r'Population of $|1\rangle$')
plt.title('Rabi oscillation')
plt.show()

We can get the corresponding cosine function by fitting these points.

In [ ]:
# Define the function to be fitted
def fit(x, a, b, c, d):
    return a * np.cos(b * x + c) + d

# Fit the curve
paraFit, _ = curve_fit(fit, ampList, popList, [-0.5, 2 * np.pi / 0.3, 0, 0.5])
def yFit(x):
    return fit(x, paraFit[0], paraFit[1], paraFit[2], paraFit[3])
y = [yFit(x) for x in ampList]

# Plot the fitted curve
plt.plot(ampList, y)
plt.xlabel('Amplitudes (a.u.)')
plt.ylabel(r'Population of $|1\rangle$')
plt.ylim(-0.05, 1)
plt.title('Rabi oscillation')
plt.show()

As it shows, the population on $|1\rangle$ oscillates periodically with the increase of pulse amplitude. Using the fitted function, we can find the corresponding amplitudes when the population of $|1\rangle$ is 0.5 and 1, which are exactly the amplitudes of $\pi / 2$ and $\pi$ pulse.

In [ ]:
ampList = np.linspace(0, 0.4, 5000)

piAmp = []
halfPiAmp = []
for amp in ampList:
    if abs(yFit(amp) - 0.5) < 1e-3:
        halfPiAmp.append(amp)
    if abs(yFit(amp) - 0.98) < 1e-3:
        piAmp.append(amp)

# find the corresponding amplitudes
x90 = min(halfPiAmp)
x180 = min(piAmp)

# Print the results
print(f'The amplitudes of pi/2 pulse: {x90}')
print(f'The amplitudes of pi pulse: {x180}')   

## Measure relaxation time $T_1$ 

To measure $T_1$ relaxation time, we first implement a pi pulse to maximize the population on the excited state. Then we record the evolution of the population over time to obtain the relaxation time $T_1$.

In [ ]:
# Define the idling time
tIdle = 2 * t1

# Define the wave to flip the qubit state
wave = gaussian(t=tg, a=x180, tau=tau, sigma=sigma)

# Initialize a job
job = ham.createJob()

# Firstly, apply a X gate to flip the qubit
job.appendWave(operators=driveX, onSubSys=0, waves=wave)

# Then simulate the evolution during the idling time
job.appendWave(operators=driveX, onSubSys=0, waves=square(tIdle, 0))

After defining the task and the initial state, we run the simulation.

In [ ]:
# Define the initial state for the simulation
stateInit = basis(level, 0) 
    
# Run the simulation of the open system evolution
result = runHamiltonian(ham=ham, state0=stateInit, job=job, isOpen=True)

# Define the projector |1><1|
prj = basis(level, 1) @ dagger(basis(level, 1))

popList = []

# Calculate the population of |1> during the evolution
for rho in result[0]['evolution_history']:
    popList.append(expect(prj, rho))

# Get the maximum time of the job
maxTime, _ = job.computeMaxTime()

tList = np.linspace(0, maxTime, len(popList))

# Plot the time-evolution poplulation for simulation and prediction 
plt.plot(tList, popList, '-', label='simulation')
tIdleList = np.linspace(tg, tIdle, 20)
plt.plot(tIdleList, np.exp(-1. / t1 * np.array(tIdleList - tg)), label='prediction')
plt.xlabel('Time (ns)')
plt.ylabel(r'Population of $|1\rangle$')
plt.title(r'$T_1$ measurement')
plt.legend()
plt.show()

The figure above is the dynamical evolution of population of the excited state $|1\rangle$. It can be seen that within tens of nanoseconds from the beginning of the experiment, the applied pulse of the $X$ gate drives the qubit from the ground state to the excited state. Then in the following idling time, the decay rate of the excited state's population is consistent with the theoretical value. At the time $t=T_1$, the population of $|1\rangle$ decays to $1/e$.

## Measure decoherence time $T_2$  by Ramsey experiment

To measure decoherence time $T_2$, we first implement a $\pi / 2$ pulse. After a delay time $t_{\rm idle}$, we implement another $\pi / 2$ pulse again and obtain the population of $|1\rangle$. In this process, we can observe oscillation over the delay time due to the detuning between the frequency of drive pulse and qubit.

In [ ]:
# Define the maximum idling time
maxTime = t2

# Define the detuning 
detuning = 2 * pi * 8. / maxTime

# Define the job for Ramsey experiment
tList = np.linspace(0, maxTime, 200)
ramseyJob = ham.createJobList()

for t in tList:
    job = ham.createJob()
    job.appendWave(driveX, 0, gaussian(t=tg, a=x90, tau=tau, sigma=sigma), compact=False)  # pi/2 pulse
    job.appendWave(driveZ, 0, waves=square(t, detuning), compact=False)  # simulate the rotation due to the detuning
    job.appendWave(driveX, 0, gaussian(t=tg, a=-x90, tau=tau, sigma=sigma), compact=False)  # pi/2 pulse
    ramseyJob.addJob(job)

After defining the list of jobs and initial state, we run the simulation.

In [ ]:
# Run the simulation starting with initial state |0>.
stateInit = basis(level, 0)
result = runHamiltonian(ham, stateInit, jobList=ramseyJob, isOpen=True)

Plot the population of $|1\rangle$ over the delayed time.

In [ ]:
popList = []

prj = basis(level, 1) @ dagger(basis(level, 1))

for res in result:
    rho = res['state']
    popList.append(expect(prj, rho))

plt.plot(tList, popList, '.b', label='simulation')
plt.xlabel('Delayed time (ns)')
plt.ylabel(r'Population of $|1\rangle$')
plt.legend()
plt.show()

Then we fit the curve with a defined fitting function, and estimate $T_2$ according to the parameters of the fitting function.

In [ ]:
# Define the fitting function
def fitRamsey(x, a, b):
    return - np.cos(a * x) * np.exp(- b * x) * 0.5 + 0.5

paraFit, _ = curve_fit(fitRamsey, tList, popList, [detuning, 0.])

def yFit(x):
    return fitRamsey(x, paraFit[0], paraFit[1])

# Plot the result
plt.plot(tList, popList, '.b', label='simulation')
plt.plot(tList, yFit(tList), label='fit')
plt.plot(tList, np.exp(- (1 / t2 + 1/(2 * t1)) * tList) * 0.5 + 0.5, label='prediction')
plt.xlabel('Delayed time (ns)')
plt.ylabel(r'Population of $|1\rangle$')
plt.legend()
plt.show()

# Print the estimated T_2 time
print(f'The T2-dephasing time is approximately {1 / (paraFit[1] - 1 / (2 * t1))} ns')

## Single-qubit noisy simulator at the gate level

Besides, we can also use the simulator directly at the gate level. First, we call a prepared single-qubit object of `PulseSim1Q()`.

In [ ]:
model = pulseSim1Q(dt=0.2)
ham = model.createQHamiltonian()

Then we define a set of quantum gates. Here, we firstly define a set of single-qubit gates.

In [ ]:
H(model.Q[0])
X(model.Q[0])
Y(model.Q[0])
Z(model.Q[0])
H(model.Q[0])

Then we use method `model.schedule()` to generate the corresponding pulse sequence, and call `runHamiltonian` to simulate the evolution of single-qubit with the presence of decoherence. We can visualize the process via module `plotBloch()`.

In [ ]:
job = model.schedule()

res = runHamiltonian(ham, state0=basis(3, 0), job=job, isOpen=True)

history = res[0]['evolution_history']

posList = []

for rho in history:
    rho2d = project(rho, 1, 3, 2) / np.trace(rho)
    posList.append(rho2Coordinate(rho2d))

plotBloch(posList, save=True, mode='animate')

As it shows, with the decoherence, the Bloch vector of qubit would not stay on the surface of the sphere, indicating the pure initial state has evolved to a mixed state, and the non-diagonal elements of the density matrix decayed. The system lost information due to the interaction with the environment. Therefore, the relatively small $T_1$ and $T_2$ will reduce the performance of deep quantum circuits.

## Summary

This tutorial describes how to simulate a superconducting single-qubit process considering partial noise using Quanlse and visualize the results. Users can click on this link [tutorial-single-qubit-noisy-simulator.ipynb](https://github.com/baidu/Quanlse/blob/main/Tutorial/EN/tutorial-single-qubit-noisy-simulator.ipynb) to jump to the corresponding GitHub page for this Jupyter Notebook documentation to get the relevant code, try the different parameter values for further exploring the function of the Quanlse Simulator module.

## Reference

\[1\] [Carvalho, Andre RR, et al. "Error-robust quantum logic optimization using a cloud quantum computer interface." *arXiv preprint arXiv:2010.08057* (2020).](https://arxiv.org/abs/2010.08057)

\[2\] [Krantz, Philip, et al. "A quantum engineer's guide to superconducting qubits." *Applied Physics Reviews* 6.2 (2019): 021318.](https://aip.scitation.org/doi/abs/10.1063/1.5089550)